<a href="https://colab.research.google.com/github/holtnk/ASDLeads/blob/main/ASD_Lead_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

To Run:

Windows: Ctrl + F9

Mac: Command + F9

Enter your zip code when prompted

In [1]:
pip install requests


In [2]:
pip install geopy


In [3]:
pip install pandas

In [14]:
import requests
import csv
from geopy.geocoders import Nominatim

##TODO: Convert to csv file that users can upload/edit and have program search through and use for filtering (bfore adding to csv)
NPI_LIST = ['103G00000X', '103T00000X', '103TB0200X', '103TC0700X', '103TC2200X', '103TF0000X', '103TH0004X',
            '103TH0100X', '103TM1800X', '103TP0016X', '103TP0814X', '103TS0200X', '163WP0807X', '2084B0040X',
            '2084D0003X', '2084N0402X', '2084P0800X', '2084P0804X', '261QM0855X', '282NC2000X', '364SP0807X',
            '364SP0810X']

RADIUS = 100

def collect_leads(zip_code):
    providers = search_providers_by_zip(zip_code)
    if providers:
        leads = []
        for provider in providers:
            ##if provider['number'] in NPI_LIST:
            lead = {
                'Contact Name': provider.get('basic', {}).get('name'),
                'NPI': provider.get('number'),
                'Address': provider.get('addresses', [{}])[0].get('address_1'),
                'City': provider.get('addresses', [{}])[0].get('city'),
                'State': provider.get('addresses', [{}])[0].get('state'),
                'Zip': provider.get('addresses', [{}])[0].get('postal_code'),
                'Phone': provider.get('addresses', [{}])[0].get('telephone_number'),
                'Taxonomy Codes': ', '.join([taxonomy.get('code') for taxonomy in provider.get('taxonomies', [])])
            }
            print(lead) ##TEMP
            leads.append(lead)

        # Write leads to CSV
        with open('Leads.csv', 'w', newline='', encoding='utf-8') as csvfile:
            fieldnames = ['Contact Name', 'NPI', 'Address', 'City', 'State', 'Zip', 'Phone', 'Taxonomy Codes']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(leads)
        print(f"Sales leads saved to Leads.csv")
    else:
        print("No providers found or error in fetching data.")

def search_providers_by_zip(zip_code):
    base_url = 'https://npiregistry.cms.hhs.gov/api/'
    url = f'{base_url}?postal_code={zip_code}&limit=200&radius={RADIUS}&version=2.1'
    response = requests.get(url)

    if response.status_code == 200:
        data = response.json()
        results = data.get('results', [])

        return results
    else:
        print(f'Error fetching data. Status code: {response.status_code}')
        return None

if __name__ == "__main__":
    zip_code = input("Enter your zip code: ")
    print(f"Searching for providers within {RADIUS} miles of {zip_code}")

    collect_leads(zip_code)

Enter your zip code: 98144
Searching for providers within 100 miles of 98144
{'Contact Name': None, 'NPI': '1124890215', 'Address': '1410 S COLUMBIAN WAY APT 107', 'City': 'SEATTLE', 'State': 'WA', 'Zip': '981447326', 'Phone': '206-472-4302', 'Taxonomy Codes': '171M00000X'}
{'Contact Name': None, 'NPI': '1306563739', 'Address': '1200 12TH AVE S STE 901', 'City': 'SEATTLE', 'State': 'WA', 'Zip': '981442712', 'Phone': '206-548-3114', 'Taxonomy Codes': '363LF0000X'}
{'Contact Name': None, 'NPI': '1013525633', 'Address': '5923 39TH AVE S', 'City': 'SEATTLE', 'State': 'WA', 'Zip': '981182616', 'Phone': '206-468-6406', 'Taxonomy Codes': '163W00000X'}
{'Contact Name': None, 'NPI': '1295340586', 'Address': '2345 RAINIER AVE S', 'City': 'SEATTLE', 'State': 'WA', 'Zip': '981445348', 'Phone': '206-325-5725', 'Taxonomy Codes': '183500000X'}
{'Contact Name': None, 'NPI': '1669070520', 'Address': '3401 RAINIER AVE S', 'City': 'SEATTLE', 'State': 'WA', 'Zip': '981446954', 'Phone': None, 'Taxonomy Cod

In [ ]:
from geopy.distance import distance
from geopy.geocoders import Nominatim

def calculate_distance(zip1, zip2):
    geolocator = Nominatim(user_agent="zipcode_distance_calculator")

    try:
        location1 = geolocator.geocode(f"{zip1}, USA")
        location2 = geolocator.geocode(f"{zip2}, USA")

        if not location1:
            raise ValueError(f"Cannot find location for zip code {zip1}")
        if not location2:
            raise ValueError(f"Cannot find location for zip code {zip2}")

        # Extract coordinates from location objects
        coords1 = (location1.latitude, location1.longitude)
        coords2 = (location2.latitude, location2.longitude)

        # Calculate distance between the coordinates
        dist = distance(coords1, coords2).miles  # Use .kilometers for kilometers

        return dist

    except Exception as e:
        print(f"Error: {e}")
        return None

print(calculate_distance("98021", "98144"))


14.767407180707854
